In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os, sys
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from deepomics import neuralnetwork as nn
from deepomics import utils, fit, visualize, saliency, metrics

import helper

np.random.seed(247)
tf.set_random_seed(247)

In [ ]:
data_path = '../data/Synthetic_dataset.h5'
results_path = '../results'
params_path = utils.make_directory(results_path, 'model_params')

# load dataset
train, valid, test = helper.load_synthetic_dataset(data_path)

test_model = helper.load_synthetic_models(data_path, dataset='test')
    
# get data shapes
input_shape = list(train['inputs'].shape)
input_shape[0] = None
output_shape = [None, train['targets'].shape[1]]

In [ ]:
tf.reset_default_graph()

model_name = 'DistNet'
dropout_status = True
l2_status = True
bn_status = True
    
# load model parameters
model_layers, optimization, genome_model = helper.load_model(model_name, 
                                                             input_shape, 
                                                             dropout_status, 
                                                             l2_status, 
                                                             bn_status)

# build neural network class
nnmodel = nn.NeuralNet(seed=247)
nnmodel.build_layers(model_layers, optimization, supervised=True)

# compile neural trainers
model_path = utils.make_directory(params_path, model_name)
file_path = os.path.join(model_path, model_name+'_do_l2_bn')
nntrainer = nn.NeuralTrainer(nnmodel, save='best', file_path=file_path)

# initialize session
sess = utils.initialize_session()

# load best parameters
nntrainer.set_best_parameters(sess)

activations = nntrainer.get_activations(sess, test, layer='output')

In [ ]:

target = 0.9865 # 9865  9815

right_index = np.where(test['targets'][:,0]==1)[0]
index = right_index[np.argmin((activations[right_index] - target)**2)]

X = np.expand_dims(test['inputs'][index], axis=0)
X_model = test_model[index]


fig = plt.figure(figsize=(10,4))
plt.subplot(3,1,1)

model_name = 'LocalNet'
model_path = utils.make_directory(params_path, model_name)
file_path = os.path.join(model_path, model_name+'_do_l2_bn')

# saliency parameters
params = {'model_name': model_name, 
          'input_shape': input_shape, 
          'dropout_status': dropout_status,
          'l2_status': l2_status,
          'bn_status': bn_status,
          'model_path': file_path+'_best.ckpt',
         }


guided_saliency = helper.backprop(X, layer='output', class_index=0, params=params, method='guided')
visualize.plot_seq_pos_saliency(np.squeeze(X).T, np.squeeze(guided_saliency[0]).T, alphabet='dna', nt_width=50, norm_factor=3)

plt.subplot(3,1,2)
model_name = 'DistNet'
model_path = utils.make_directory(params_path, model_name)
file_path = os.path.join(model_path, model_name+'_do_l2_bn')

# saliency parameters
params = {'model_name': model_name, 
          'input_shape': input_shape, 
          'dropout_status': dropout_status,
          'l2_status': l2_status,
          'bn_status': bn_status,
          'model_path': file_path+'_best.ckpt',
         }


guided_saliency = helper.backprop(X, layer='output', class_index=0, params=params, method='guided')
visualize.plot_seq_pos_saliency(np.squeeze(X).T, np.squeeze(guided_saliency[0]).T, alphabet='dna', nt_width=50, norm_factor=3)


plt.subplot(3,1,3)
visualize.plot_seq_pos_saliency(np.squeeze(X).T, X_model, alphabet='dna', nt_width=50, norm_factor=3)

#output_name = str(index)+'_'+name+'_{:.2f}'.format(p[i]) + '_' + '{:.2f}'.format(y[i])
#outfile = os.path.join(plot_path, output_name+'.pdf')
#fig.savefig(outfile, format='pdf', dpi=200, bbox_inches='tight')

In [ ]:
tf.reset_default_graph()

model_name = 'LocalNet'
dropout_status = True
l2_status = True
bn_status = True

# load model parameters
model_layers, optimization, genome_model = helper.load_model(model_name, 
                                                             input_shape, 
                                                             dropout_status, 
                                                             l2_status, 
                                                             bn_status)
# build neural network class
nnmodel = nn.NeuralNet(seed=247)
nnmodel.build_layers(model_layers, optimization, supervised=True)

# compile neural trainers
model_path = utils.make_directory(params_path, model_name)
file_path = os.path.join(model_path, model_name+'_do_l2_bn')
nntrainer = nn.NeuralTrainer(nnmodel, save='best', file_path=file_path)

# initialize session
sess = utils.initialize_session()

# load best parameters
nntrainer.set_best_parameters(sess)


# Layer 1

In [ ]:
# saliency parameters
params = {'model_name': model_name, 
          'input_shape': input_shape, 
          'dropout_status': dropout_status,
          'l2_status': l2_status,
          'bn_status': bn_status,
          'model_path': file_path+'_best.ckpt',
         }


for neuron_index in range(24):
    val = helper.backprop(X, layer='conv1d_0_active', class_index=neuron_index, params=params, method='guided')
    saliency.append(val)
    
    plt.figure(figsize=(100,10))
    pwm = utils.normalize_pwm(np.squeeze(val[0]).T, factor=3)
    logo = visualize.seq_logo(pwm, height=500, nt_width=100, norm=0, alphabet='dna')
    plt.imshow(logo)
    plt.title(str(neuron_index), fontsize=100)


In [ ]:
# saliency parameters
params = {'model_name': model_name, 
          'input_shape': input_shape, 
          'dropout_status': dropout_status,
          'l2_status': l2_status,
          'bn_status': bn_status,
          'model_path': file_path+'_best.ckpt',
         }


active_indices = [8, 9, 17, 21]
num_plots = len(active_indices)

saliency = []
for neuron_index in active_indices:
    val = helper.backprop(X, layer='conv1d_0_active', class_index=neuron_index, params=params, method='guided')
    saliency.append(val)
    

    
fig = plt.figure(figsize=(100,10))
plt.subplot(num_plots,1,1)
pwm = utils.normalize_pwm(np.squeeze(saliency[0]).T, factor=3)
logo = visualize.seq_logo(pwm, height=500, nt_width=100, norm=0, alphabet='dna')
plt.imshow(logo)
plt.xticks([])
plt.yticks([])
plt.ylabel(str(active_indices[0]), fontsize=70)

plt.subplot(num_plots,1,2)
pwm = utils.normalize_pwm(np.squeeze(saliency[1]).T, factor=3)
logo = visualize.seq_logo(pwm, height=500, nt_width=100, norm=0, alphabet='dna')
plt.imshow(logo)
plt.xticks([])
plt.yticks([])
plt.ylabel(str(active_indices[1]), fontsize=70)

plt.subplot(num_plots,1,3)
pwm = utils.normalize_pwm(np.squeeze(saliency[2]).T, factor=3)
logo = visualize.seq_logo(pwm, height=500, nt_width=100, norm=0, alphabet='dna')
plt.imshow(logo)
plt.xticks([])
plt.yticks([])
plt.ylabel(str(active_indices[2]), fontsize=70)


plt.subplot(num_plots,1,4)
pwm = utils.normalize_pwm(np.squeeze(saliency[3]).T, factor=3)
logo = visualize.seq_logo(pwm, height=500, nt_width=100, norm=0, alphabet='dna')
plt.imshow(logo)
plt.xticks([])
plt.yticks([])
plt.ylabel(str(active_indices[3]), fontsize=70)

save_path = utils.make_directory(results_path, 'layers')
outfile = os.path.join(save_path,model_name+'_layer1.pdf')
fig.savefig(outfile, format='pdf', dpi=200, bbox_inches='tight')
        

# Layer 2

In [ ]:
nnmodel.inspect_layers()

In [ ]:
tf.reset_default_graph()

model_name = 'LocalNet'
dropout_status = True
l2_status = True
bn_status = True

# model save path
    
# load model parameters
model_layers, optimization, genome_model = helper.load_regulatory_code_model(model_name, input_shape, output_shape,
                                               dropout_status, l2_status, bn_status)

# build neural network class
nnmodel = nn.NeuralNet(seed=247)
nnmodel.build_layers(model_layers, optimization, supervised=True)

# compile neural trainers
model_path = utils.make_directory(params_path, model_name)
file_path = os.path.join(model_path, model_name+'_do_l2_bn')
nntrainer = nn.NeuralTrainer(nnmodel, save='best', file_path=file_path)

# initialize session
sess = utils.initialize_session()

# load best parameters
nntrainer.set_best_parameters(sess)

activations = nntrainer.get_activations(sess, {'inputs': X}, layer='conv1d_1_active')

In [ ]:
active_indices = np.argsort(np.squeeze(activations))[::-1]

In [ ]:
# saliency parameters
params = {'model_name': model_name, 
          'input_shape': input_shape, 
          'dropout_status': dropout_status,
          'l2_status': l2_status,
          'bn_status': bn_status,
          'model_path': file_path+'_best.ckpt',
         }

saliency = []
for neuron_index in active_indices[40:50]:
    val = helper.backprop(X, layer='conv1d_1_active', class_index=neuron_index, params=params, method='guided')
    saliency.append(val)
    
    plt.figure(figsize=(100,10))
    pwm = utils.normalize_pwm(np.squeeze(val[0]).T, factor=3)
    logo = visualize.seq_logo(pwm, height=500, nt_width=100, norm=0, alphabet='dna')
    plt.imshow(logo)
    plt.title(str(neuron_index), fontsize=100)


In [ ]:
active_indices = [5, 44, 55, 66]
num_plots = len(active_indices)

saliency = []
for neuron_index in active_indices:
    val = helper.backprop(X, layer='conv1d_1_active', class_index=neuron_index, params=params, method='guided')
    saliency.append(val)
    

In [ ]:
fig = plt.figure(figsize=(100,10))
plt.subplot(num_plots,1,1)
pwm = utils.normalize_pwm(np.squeeze(saliency[0]).T, factor=3)
logo = visualize.seq_logo(pwm, height=500, nt_width=100, norm=0, alphabet='dna')
plt.imshow(logo)
plt.xticks([])
plt.yticks([])
plt.ylabel(str(active_indices[0]), fontsize=70)

plt.subplot(num_plots,1,2)
pwm = utils.normalize_pwm(np.squeeze(saliency[1]).T, factor=3)
logo = visualize.seq_logo(pwm, height=500, nt_width=100, norm=0, alphabet='dna')
plt.imshow(logo)
plt.xticks([])
plt.yticks([])
plt.ylabel(str(active_indices[1]), fontsize=70)

plt.subplot(num_plots,1,3)
pwm = utils.normalize_pwm(np.squeeze(saliency[2]).T, factor=3)
logo = visualize.seq_logo(pwm, height=500, nt_width=100, norm=0, alphabet='dna')
plt.imshow(logo)
plt.xticks([])
plt.yticks([])
plt.ylabel(str(active_indices[2]), fontsize=70)


plt.subplot(num_plots,1,4)
pwm = utils.normalize_pwm(np.squeeze(saliency[3]).T, factor=3)
logo = visualize.seq_logo(pwm, height=500, nt_width=100, norm=0, alphabet='dna')
plt.imshow(logo)
plt.xticks([])
plt.yticks([])
plt.ylabel(str(active_indices[3]), fontsize=70)

save_path = utils.make_directory(results_path, 'layers')
outfile = os.path.join(save_path,model_name+'_layer2_2.pdf')
fig.savefig(outfile, format='pdf', dpi=200, bbox_inches='tight')
        

# output layer

In [ ]:
model_path = utils.make_directory(params_path, model_name)
file_path = os.path.join(model_path, model_name+'_do_l2_bn')

# saliency parameters
params = {'model_name': model_name, 
          'input_shape': input_shape, 
          'dropout_status': dropout_status,
          'l2_status': l2_status,
          'bn_status': bn_status,
          'model_path': file_path+'_best.ckpt',
         }

fig = plt.figure(figsize=(100,2))
guided_saliency = helper.backprop(X, layer='output', class_index=0, params=params, method='guided')

pwm = utils.normalize_pwm(np.squeeze(guided_saliency[0]).T, factor=3)
logo = visualize.seq_logo(pwm, height=500, nt_width=100, norm=0, alphabet='dna')
plt.imshow(logo)
plt.xticks([])
plt.yticks([])
plt.ylabel('Output', fontsize=38)

save_path = utils.make_directory(results_path, 'layers')
outfile = os.path.join(save_path,model_name+'_output.pdf')
fig.savefig(outfile, format='pdf', dpi=200, bbox_inches='tight')



In [ ]:
fig = plt.figure(figsize=(100,4))
visualize.plot_seq_pos_saliency(np.squeeze(X).T, X_model, alphabet='dna', nt_width=50, norm_factor=3)
save_path = utils.make_directory(results_path, 'layers')
outfile = os.path.join(save_path,model_name+'_model.pdf')
fig.savefig(outfile, format='pdf', dpi=200, bbox_inches='tight')
